# Friendship Paradox Twitter

-------------------------------------------------

The raw code for this Jupyter notebook is by default hidden for easier reading. The main focus of this particular page of the notebook is on the graphs and their interpretation. To toggle on/off the raw code, click below:

In [1]:
# Setup Code toggle button
from IPython.core.display import HTML  

HTML(''' 
<center><h3>
<a href="javascript:code_toggle()">Talk is cheap, show me the code.</a>
</center></h3>
<script>
    var code_show=true; //true -> hide code at first

    function code_toggle() {
        $('div.prompt').hide(); // always hide prompt

        if (code_show){
            $('div.input').hide();
        } else {
            $('div.input').show();
        }
        code_show = !code_show
    }
    $( document ).ready(code_toggle);
</script>
''')

In [2]:
# Setup notebook theme
from jupyterthemes import get_themes
from jupyterthemes.stylefx import set_nb_theme
set_nb_theme(get_themes()[1])

In [3]:
# Load R magic
%load_ext rpy2.ipython

## Get the Data

For a list of `python-twitter` `User` class attributes see: [https://python-twitter.readthedocs.io/en/latest/_modules/twitter/models.html#User.NewFromJsonDict](https://python-twitter.readthedocs.io/en/latest/_modules/twitter/models.html#User.NewFromJsonDict).

In [4]:
from followers import Followers

uid = 151901549
f = Followers(uid)
followers = f.api.GetFollowers(uid)

followers

[User(ID=827268804359053313, ScreenName=mmicros1),
 User(ID=789293970677456896, ScreenName=74paddycakes),
 User(ID=799272100599046145, ScreenName=digitopia_nl),
 User(ID=203096228, ScreenName=PruittAL),
 User(ID=81650379, ScreenName=brendaberkelaar),
 User(ID=47899646, ScreenName=CassPF),
 User(ID=899863316, ScreenName=OKMotovlog),
 User(ID=30802131, ScreenName=mekonkol),
 User(ID=761396027446398976, ScreenName=Stesker3D),
 User(ID=825218681252765696, ScreenName=HussamHallak1),
 User(ID=20255236, ScreenName=carolc16),
 User(ID=825049390998024192, ScreenName=printuu),
 User(ID=733536950963994624, ScreenName=dtscnc),
 User(ID=737852660183932928, ScreenName=Tunaz_Islam),
 User(ID=9630082, ScreenName=MeredithA),
 User(ID=258599525, ScreenName=MeghanHoyer),
 User(ID=3122708646, ScreenName=AntonJRasmussen),
 User(ID=1634706151, ScreenName=LuytenBram),
 User(ID=399805362, ScreenName=MohdKamal6),
 User(ID=269424738, ScreenName=pmyoung84),
 User(ID=810540893396733953, ScreenName=Karki693),
 Use

In [5]:
for follower in followers:
    print("User {} has {} followers."\
          .format(follower.screen_name, follower.followers_count))

User mmicros1 has 2 followers.
User 74paddycakes has 2 followers.
User digitopia_nl has 17 followers.
User PruittAL has 408 followers.
User brendaberkelaar has 199 followers.
User CassPF has 2583 followers.
User OKMotovlog has 176 followers.
User mekonkol has 652 followers.
User Stesker3D has 217 followers.
User HussamHallak1 has 0 followers.
User carolc16 has 15 followers.
User printuu has 48 followers.
User dtscnc has 140 followers.
User Tunaz_Islam has 17 followers.
User MeredithA has 9606 followers.
User MeghanHoyer has 1721 followers.
User AntonJRasmussen has 5 followers.
User LuytenBram has 249 followers.
User MohdKamal6 has 87 followers.
User pmyoung84 has 340 followers.
User Karki693 has 21 followers.
User BL_Labs has 4280 followers.
User IntelligenceTV has 231951 followers.
User maheedhargunnam has 1 followers.
User nattiyak has 626 followers.
User marcinwilkowski has 724 followers.
User minornotez has 11 followers.
User katestarbird has 3966 followers.
User ACMSIGIR has 475 f

In [6]:
with open('followers.csv', 'w') as outfile:
    outfile.write('user,followers\nphonedude_mln,625\n')
    for follower in followers:
        outfile.write('{},{}\n'.format(follower.screen_name, follower.followers_count))   

&nbsp;

## Get Data into R

&nbsp;

In [7]:
%%R

# Tell R to be quiet
options(warn=-1)

library(jsonlite)
library(igraph)

# Get the data
df <- read.csv('followers.csv')
df

/usr/local/lib/python3.4/dist-packages/rpy2/rinterface/__init__.py:186: RRuntimeWarning: 
Attache Paket: ‘igraph’


  warnings.warn(x, RRuntimeWarning)
/usr/local/lib/python3.4/dist-packages/rpy2/rinterface/__init__.py:186: RRuntimeWarning: The following objects are masked from ‘package:stats’:

    decompose, spectrum


  warnings.warn(x, RRuntimeWarning)
/usr/local/lib/python3.4/dist-packages/rpy2/rinterface/__init__.py:186: RRuntimeWarning: The following object is masked from ‘package:base’:

    union


  warnings.warn(x, RRuntimeWarning)


               user followers
1     phonedude_mln       625
2          mmicros1         2
3      74paddycakes         2
4      digitopia_nl        17
5          PruittAL       408
6   brendaberkelaar       199
7            CassPF      2583
8        OKMotovlog       176
9          mekonkol       652
10        Stesker3D       217
11    HussamHallak1         0
12         carolc16        15
13          printuu        48
14           dtscnc       140
15      Tunaz_Islam        17
16        MeredithA      9606
17      MeghanHoyer      1721
18  AntonJRasmussen         5
19       LuytenBram       249
20       MohdKamal6        87
21        pmyoung84       340
22         Karki693        21
23          BL_Labs      4280
24   IntelligenceTV    231951
25  maheedhargunnam         1
26         nattiyak       626
27  marcinwilkowski       724
28       minornotez        11
29     katestarbird      3966
30         ACMSIGIR       475
31         claussni       194
32  njcomputergroup      1727
33        

&nbsp;

### Some Stats

&nbsp;

In [48]:
%%R

cat("Standard Deviation: ")
cat(sd(na.omit(df$followers)))

Standard Deviation: 10143.41

In [8]:
%%R

summary(df$followers)

    Min.  1st Qu.   Median     Mean  3rd Qu.     Max. 
     0.0     63.0    310.0   1508.0    973.5 232000.0 


&nbsp;

## Graph

&nbsp;

In [9]:
%%R

# Get just friend data
df <- na.omit(df[with(df, order(followers)), ])
df

               user followers
11    HussamHallak1         0
86      CRajaharsha         0
551    ahmedkalka97         0
569  witch_doctor81         0
574       unknown_6         0
25  maheedhargunnam         1
223    WolframElvis         1
241          hdo003         1
445       amaranaas         1
539         Milbala         1
552  DwiDwisyafitri         1
2          mmicros1         2
3      74paddycakes         2
160    Shaaban_Migo         2
253    KyleFJackson         2
339      DrAinenwar         2
380         lnsails         2
63       Jack_Stran         3
92        mridulish         3
96    zhaohuan_wang         3
116       amlhasbat         3
131 Mittrach_garten         3
137   KarenKvaughan         3
364         meterz1         3
474         chglass         3
40          es_land         4
128 Fernando___Melo         4
132      ArnoudGoos         4
175       CrowNaito         4
264        katjakra         4
280     JS_Smith_71         4
18  AntonJRasmussen         5
108       

In [47]:
%%R

library(plotly)

xs <- seq.int(1, length(df$followers))
ys <- df$followers
colors <- ifelse(df$user == 'phonedude_mln', '#DE2D26', '#CCCCCC')

title_font <- list(
    family = "Courier New, monospace",
    size = 70,
    color = "666666"
)

axis_font <- list(
    family = "Courier New, monospace",
    size = 50,
    color = "666666"
)

tick_font <- list(
    family = "Courier New, monospace",
    size = 30
)

margin <- list(
    l = 200,
    r = 200,
    b = 200,
    t = 200,
    pad = 8
)

p <- plot_ly() %>%
    add_trace(
        x = xs,
        y = ys,
        type = "bar",
        name = "Followers",
        marker = list(color = colors),
        text = ~paste("User: ", df$user)
    ) %>% layout(
        title = "Twitter Followers",
        font = title_font,
        xaxis = list(
            title = "Users",
            font = axis_font,
            tickfont = tick_font
        ),
        yaxis = list(
            title = "Followers",
            font = axis_font,
            tickfont = tick_font
        ),
        autosize = F,
        resolution = 300,
        width = 2556,
        height = 2556,
        margin = margin
    )

plotly_IMAGE(
    p, 
    format = "png",
    out_file = "followers_full.png",
    width=2556,
    height=2556,
    fileopt = "overwrite"
)

![followers_full.png](followers_full.png)

In [44]:
%%R

p <- plot_ly() %>%
    add_trace( # Trick Plotly into adding the colored bar to the legend
            x = c(0),
            y = c(0),
            mode = "bar",
            name = "Dr. Nelson",
            marker = list(color = '#DE2D26')
    ) %>% add_trace(
        x = xs,
        y = ys,
        type = "bar",
        name = "followers",
        marker = list(color = colors),
        text = ~paste("User: ", df$user)
    ) %>% add_trace(
        x = seq.int(1, length(xs)),
        y = rep(1508, length(xs)),
        type = "scatter",
        mode = "lines",
        name = "Mean",
        line = list(color = '#0C63AB'),
        hoverinfo = "text",
        text = ~paste("Mean: 1508")
    ) %>% add_trace(
        x = seq.int(1, length(xs)),
        y = rep(310, length(xs)),
        type = "scatter",
        mode = "lines",
        name = "Median",
        line = list(color = '#4C4C4C'),
        hoverinfo = "text",
        text = ~paste("Median: 310")
    ) %>% layout(
        title = "Twitter Followers",
        font = title_font,
        xaxis = list(
            title = "Users",
            font = axis_font,
            tickfont = tick_font,
            range = c(290, 610)
        ),
        yaxis = list(
            title = "Followers",
            font = axis_font,
            tickfont = tick_font,
            range = c(0, 8000)
        ),
        legend = list(
            x = 0.1,
            y = 0.9,
            bgcolor = "#E2E2E2",
            bordercolor = "#666666",
            borderwidth=2
        ),
        autosize = F,
        resolution = 300,
        width = 2556,
        height = 2556,
        margin = margin
    )

plotly_IMAGE(
    p, 
    format = "png",
    out_file = "followers_half.png",
    width=2556,
    height=2556,
    fileopt = "overwrite"
)

/usr/local/lib/python3.4/dist-packages/rpy2/rinterface/__init__.py:186: RRuntimeWarning: A marker object has been specified, but markers is not in the mode
Adding markers to the mode...

  warnings.warn(x, RRuntimeWarning)


![followers_half.png](followers_half.png)

In [45]:
%%R

q <- plot_ly() %>%
    add_trace( # Trick Plotly into adding the colored bar to the legend
        x = c(0),
        y = c(0),
        mode = "bar",
        name = "Dr. Nelson",
        marker = list(color = '#DE2D26')
    ) %>% add_trace(
        x = xs,
        y = ys,
        type = "bar",
        name = "followers",
        marker = list(color = colors),
        text = ~paste("User: ", df$user)
    ) %>% add_trace(
        x = seq.int(1, length(xs)),
        y = rep(1508, length(xs)),
        type = "scatter",
        mode = "lines",
        name = "Mean",
        line = list(color = '#0C63AB'),
        hoverinfo = "text",
        text = ~paste("Mean: 1508")
    ) %>% add_trace(
        x = seq.int(1, length(xs)),
        y = rep(310, length(xs)),
        type = "scatter",
        mode = "lines",
        name = "Median",
        line = list(color = '#4C4C4C'),
        hoverinfo = "text",
        text = ~paste("Median: 310")
    ) %>% layout(
        title = "Twitter Followers",
        xaxis = list(
            title = "Users"
        ),
        yaxis = list(
            title = "Followers"
        )
    )

htmlwidgets::saveWidget(q, "followers.html")